In [ ]:
pip install numpy opencv-python


In [ ]:
import numpy as np
import cv2
import time

# Constants
FRAME_WIDTH = 600
FRAME_HEIGHT = 400
CALIBRATION_TIME = 30
BG_WEIGHT = 0.5
OBJ_THRESHOLD = 25

# Region of interest
region_top = 0
region_bottom = int(2 * FRAME_HEIGHT / 3)
region_left = int(FRAME_WIDTH / 2)
region_right = FRAME_WIDTH

# Variables
background = None
hand = None
frames_elapsed = 0
sos_sequence = []

# Timing for SOS message
SOS_TIME_LIMIT = 10  # seconds between gestures for SOS detection

class HandData:
    def __init__(self, top, bottom, left, right, centerX):
        self.top = top
        self.bottom = bottom
        self.left = left
        self.right = right
        self.centerX = centerX
        self.prevCenterX = 0
        self.isInFrame = False
        self.isWaving = False
        self.fingers = 0

    def update(self, top, bottom, left, right):
        self.top = top
        self.bottom = bottom
        self.left = left
        self.right = right

    def check_for_waving(self, centerX):
        self.prevCenterX = self.centerX
        self.centerX = centerX
        self.isWaving = abs(self.centerX - self.prevCenterX) > 3


def write_on_image(frame):
    global hand
    text = "Searching..."
    if frames_elapsed < CALIBRATION_TIME:
        text = "Calibrating..."
    elif hand is None or not hand.isInFrame:
        text = "No hand detected"
    else:
        if hand.isWaving:
            text = "Waving"
        elif hand.fingers == 0:
            text = "Rock"
        elif hand.fingers == 1:
            text = "Pointing"
        elif hand.fingers == 2:
            text = "Scissors"
        else:
            text = f"Fingers: {hand.fingers}"

    cv2.putText(frame, text, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)


def get_region(frame):
    region = frame[region_top:region_bottom, region_left:region_right]
    region = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
    region = cv2.GaussianBlur(region, (7, 7), 0)
    return region


def get_average(region):
    global background
    if background is None:
        background = region.copy().astype("float")
    else:
        cv2.accumulateWeighted(region, background, BG_WEIGHT)


def segment(region):
    global hand
    diff = cv2.absdiff(background.astype("uint8"), region)
    thresholded = cv2.threshold(diff, OBJ_THRESHOLD, 255, cv2.THRESH_BINARY)[1]
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        if hand:
            hand.isInFrame = False
        return None

    segmented = max(contours, key=cv2.contourArea)
    if hand:
        hand.isInFrame = True
    return thresholded, segmented


def count_fingers(thresholded_image, segmented_image):
    hull = cv2.convexHull(segmented_image, returnPoints=False)
    if len(hull) < 3:
        return 0

    defects = cv2.convexityDefects(segmented_image, hull)
    if defects is None:
        return 0

    finger_count = 0
    for i in range(defects.shape[0]):
        s, e, f, d = defects[i, 0]
        start = tuple(segmented_image[s][0])
        end = tuple(segmented_image[e][0])
        far = tuple(segmented_image[f][0])

        a = np.linalg.norm(np.array(start) - np.array(end))
        b = np.linalg.norm(np.array(start) - np.array(far))
        c = np.linalg.norm(np.array(end) - np.array(far))

        angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))

        if angle <= np.pi / 2:
            finger_count += 1

    return finger_count


def get_hand_data(thresholded_image, segmented_image):
    global hand
    convexHull = cv2.convexHull(segmented_image)
    top = tuple(convexHull[convexHull[:, :, 1].argmin()][0])
    bottom = tuple(convexHull[convexHull[:, :, 1].argmax()][0])
    left = tuple(convexHull[convexHull[:, :, 0].argmin()][0])
    right = tuple(convexHull[convexHull[:, :, 0].argmax()][0])
    centerX = int((left[0] + right[0]) / 2)

    if hand is None:
        hand = HandData(top, bottom, left, right, centerX)
    else:
        hand.update(top, bottom, left, right)

    if frames_elapsed % 6 == 0:
        hand.check_for_waving(centerX)

    hand.fingers = count_fingers(thresholded_image, segmented_image)


def check_for_sos(gesture_time):
    global sos_sequence
    current_time = time.time()

    # Keep gestures in sequence
    if len(sos_sequence) >= 3:
        sos_sequence.pop(0)

    # Record gesture with timestamp
    sos_sequence.append((gesture_time, hand.fingers))

    # Detect SOS based on the sequence
    if len(sos_sequence) == 3:
        first_time, first_fingers = sos_sequence[0]
        second_time, second_fingers = sos_sequence[1]
        third_time, third_fingers = sos_sequence[2]

        if (first_fingers == 0 and second_fingers == 5 and third_fingers == 0) and (current_time - first_time < SOS_TIME_LIMIT):
            print("SOS Sent!")
            return True

    return False


capture = cv2.VideoCapture(0)
if not capture.isOpened():
    print("Error: Could not open video device.")
    exit()

while True:
    ret, frame = capture.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    frame = cv2.resize(frame, (FRAME_WIDTH, FRAME_HEIGHT))
    frame = cv2.flip(frame, 1)
    region = get_region(frame)

    if frames_elapsed < CALIBRATION_TIME:
        get_average(region)
    else:
        region_pair = segment(region)
        if region_pair:
            thresholded, segmented = region_pair
            cv2.drawContours(frame, [segmented], -1, (0, 255, 0), 2)
            cv2.imshow("Thresholded", thresholded)
            get_hand_data(thresholded, segmented)

            # Check for SOS gesture
            if check_for_sos(time.time()):
                cv2.putText(frame, "SOS Sent!", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    write_on_image(frame)
    cv2.imshow("Hand Gesture Recognition", frame)
    frames_elapsed += 1

    if cv2.waitKey(1) & 0xFF == ord('x'):
        break

capture.release()
cv2.destroyAllWindows()
